## Machine Learning

En este punto mi modelo recibira  el id de un juego y retornara 5 juegos similares, para esto se nos recomendo en la clase de consultas un estandar muy comun llamado la similitud de coseno el cual se basa en determinar la similitud entre dos vectores previamente tratados en los pasos anteriores y terminados de organizar en este notebook.

In [1]:
import pandas as pd
import numpy as np
from numpy import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings("ignore")
pd.options.display.max_columns = None # Opcion para mostrar todas las columnas de un dataframe

Importo todos los datos que ya preprocese en el ETL

In [2]:
dfGames = pd.read_csv("Data/dfOutputSteamGames")
dfGames.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28785 entries, 0 to 28784
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    28785 non-null  int64  
 1   publisher     23985 non-null  object 
 2   genres        28785 non-null  object 
 3   title         28785 non-null  object 
 4   specs         28785 non-null  object 
 5   price         28785 non-null  float64
 6   early_access  28785 non-null  bool   
 7   id            28785 non-null  int64  
 8   developer     28785 non-null  object 
 9   release_year  28785 non-null  float64
dtypes: bool(1), float64(2), int64(2), object(5)
memory usage: 2.0+ MB


In [3]:
dfReviews = pd.read_parquet("Data/dfAustralianUserReviews")
dfReviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59305 entries, 0 to 59304
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             59305 non-null  object
 1   item_id             59305 non-null  int32 
 2   recommend           59305 non-null  bool  
 3   review              59305 non-null  object
 4   posted_year         59305 non-null  int32 
 5   title               59305 non-null  object
 6   developer           59305 non-null  object
 7   sentiment_analysis  59305 non-null  int64 
dtypes: bool(1), int32(2), int64(1), object(4)
memory usage: 2.8+ MB


In [4]:
dfAuserItems = pd.read_parquet("Data/dfAuserItems")
dfAuserItems.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2803737 entries, 0 to 5153207
Data columns (total 11 columns):
 #   Column            Dtype  
---  ------            -----  
 0   item_id           int32  
 1   item_name         object 
 2   playtime_forever  int64  
 3   playtime_2weeks   int64  
 4   steam_id          object 
 5   items_count       int64  
 6   user_id           object 
 7   genres            object 
 8   price             float64
 9   developer         object 
 10  release_year      object 
dtypes: float64(1), int32(1), int64(3), object(6)
memory usage: 246.0+ MB


In [5]:
# Unifico dos de los tres dataframes por su contenido 

mergedData = pd.merge(dfAuserItems, dfReviews[['user_id', 'posted_year', 'recommend', 'sentiment_analysis']], on='user_id', how='inner')

In [6]:
mergedData

,item_id,item_name,playtime_forever,playtime_2weeks,steam_id,items_count,user_id,genres,price,developer,release_year,posted_year,recommend,sentiment_analysis
0,10,Counter-Strike,6,0,76561197970982479,277,76561197970982479,[Action],9.99,Valve,2000,2011,True,2
1,10,Counter-Strike,6,0,76561197970982479,277,76561197970982479,[Action],9.99,Valve,2000,2011,True,2
2,10,Counter-Strike,6,0,76561197970982479,277,76561197970982479,[Action],9.99,Valve,2000,2011,True,2
3,30,Day of Defeat,7,0,76561197970982479,277,76561197970982479,[Action],4.99,Valve,2003,2011,True,2
4,30,Day of Defeat,7,0,76561197970982479,277,76561197970982479,[Action],4.99,Valve,2003,2011,True,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4262666,400040,ShareX,1,0,76561198312638244,36,76561198312638244,"[Audio Production, Design &amp; Illustration, ...",4.99,ShareX Team,2007,2014,True,2
4262667,379720,DOOM,1178,0,76561198312638244,36,76561198312638244,[Action],29.99,id Software,2016,2014,True,2
4262668,379720,DOOM,1178,0,76561198312638244,36,76561198312638244,[Action],29.99,id Software,2016,2014,True,2
4262669,379720,DOOM,1178,0,76561198312638244,36,76561198312638244,[Action],29.99,id Software,2016,2014,True,2


In [7]:
mergedData = mergedData.drop(columns=["playtime_2weeks", "steam_id"]) # Elimine innecesarias

In [8]:
mergedData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4262671 entries, 0 to 4262670
Data columns (total 12 columns):
 #   Column              Dtype  
---  ------              -----  
 0   item_id             int32  
 1   item_name           object 
 2   playtime_forever    int64  
 3   items_count         int64  
 4   user_id             object 
 5   genres              object 
 6   price               float64
 7   developer           object 
 8   release_year        object 
 9   posted_year         int32  
 10  recommend           bool   
 11  sentiment_analysis  int64  
dtypes: bool(1), float64(1), int32(2), int64(3), object(5)
memory usage: 329.3+ MB


In [9]:
# Creo una lista con las columnas especificas a tratar en un nuevo DF
groupData = ["item_id", "item_name", "playtime_forever", "items_count", "user_id", "genres", "price", "developer", "release_year", "posted_year", "recommend", "sentiment_analysis"]
filter = mergedData[groupData]
filter

,item_id,item_name,playtime_forever,items_count,user_id,genres,price,developer,release_year,posted_year,recommend,sentiment_analysis
0,10,Counter-Strike,6,277,76561197970982479,[Action],9.99,Valve,2000,2011,True,2
1,10,Counter-Strike,6,277,76561197970982479,[Action],9.99,Valve,2000,2011,True,2
2,10,Counter-Strike,6,277,76561197970982479,[Action],9.99,Valve,2000,2011,True,2
3,30,Day of Defeat,7,277,76561197970982479,[Action],4.99,Valve,2003,2011,True,2
4,30,Day of Defeat,7,277,76561197970982479,[Action],4.99,Valve,2003,2011,True,2
...,...,...,...,...,...,...,...,...,...,...,...,...
4262666,400040,ShareX,1,36,76561198312638244,"[Audio Production, Design &amp; Illustration, ...",4.99,ShareX Team,2007,2014,True,2
4262667,379720,DOOM,1178,36,76561198312638244,[Action],29.99,id Software,2016,2014,True,2
4262668,379720,DOOM,1178,36,76561198312638244,[Action],29.99,id Software,2016,2014,True,2
4262669,379720,DOOM,1178,36,76561198312638244,[Action],29.99,id Software,2016,2014,True,2


In [10]:
filter[groupData] = filter[groupData].astype(str) # Para facilitar el entrenamiento del modelo 

In [11]:
filter = filter.drop_duplicates() # Elimino duplicados

In [12]:
from tools import countNulls
countNulls(filter)

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


,Columna,Número de Nulos,Porcentaje de Nulos
item_id,item_id,0,0.0%
item_name,item_name,0,0.0%
playtime_forever,playtime_forever,0,0.0%
items_count,items_count,0,0.0%
user_id,user_id,0,0.0%
genres,genres,0,0.0%
price,price,0,0.0%
developer,developer,0,0.0%
release_year,release_year,0,0.0%
posted_year,posted_year,0,0.0%


In [13]:
from tools import mixture

In [14]:
# Cree una columna llamada "groupedata" que integra todas las demas
filter["groupedata"]=filter.apply(mixture, axis=1)
filter

,item_id,item_name,playtime_forever,items_count,user_id,genres,price,developer,release_year,posted_year,recommend,sentiment_analysis,groupedata
0,10,Counter-Strike,6,277,76561197970982479,['Action'],9.99,Valve,2000,2011,True,2,76561197970982479 Counter-Strike 277 6 Valve [...
3,30,Day of Defeat,7,277,76561197970982479,['Action'],4.99,Valve,2003,2011,True,2,76561197970982479 Day of Defeat 277 7 Valve ['...
6,300,Day of Defeat: Source,4733,277,76561197970982479,['Action'],9.99,Valve,2010,2011,True,2,76561197970982479 Day of Defeat: Source 277 47...
9,240,Counter-Strike: Source,1853,277,76561197970982479,['Action'],19.99,Valve,2004,2011,True,2,76561197970982479 Counter-Strike: Source 277 1...
12,3830,Psychonauts,333,277,76561197970982479,['Action'],9.99,Double Fine Productions,2005,2011,True,2,76561197970982479 Psychonauts 277 333 Double F...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4262651,238010,Deus Ex: Human Revolution - Director's Cut,284,36,76561198312638244,['Action' 'RPG'],19.99,"Eidos Montreal,Feral Interactive (Mac)",2013,2014,True,2,76561198312638244 Deus Ex: Human Revolution - ...
4262655,730,Counter-Strike: Global Offensive,23,36,76561198312638244,['Action'],14.99,Valve,2012,2014,True,2,76561198312638244 Counter-Strike: Global Offen...
4262659,362890,Black Mesa,2184,36,76561198312638244,['Action' 'Adventure' 'Indie' 'Early Access'],19.99,Crowbar Collective,2015,2014,True,2,76561198312638244 Black Mesa 36 2184 Crowbar C...
4262663,400040,ShareX,1,36,76561198312638244,['Audio Production' 'Design &amp; Illustration...,4.99,ShareX Team,2007,2014,True,2,76561198312638244 ShareX 36 1 ShareX Team ['Au...


In [15]:
uniqueValues = filter['item_id'].unique()
uniqueValues

array(['10', '30', '300', ..., '405960', '464120', '352760'], dtype=object)

In [ ]:
for value in uniqueValues:
    print( ' '.join(value))

In [17]:


'''

Creamos una nueva columna llamada "datos_agrupado" en el DataFrame "filter".
Esta columna contiene una concatenación de los valores de todas las columnas en cada fila.

Creamos el modelo de vectorización y la matriz
 
model = TfidfVectorizer(stop_words='english')
womb = model.fit_transform(filter[groupData])

Comentario:
Creamos un modelo de vectorización TF-IDF usando TfidfVectorizer de scikit-learn.
Este modelo se ajusta a los datos del DataFrame "filter" en la columna "groupData".
La matriz resultante se almacena en la variable "womb".
La vectorización TF-IDF convierte el texto en una matriz numérica basada en la frecuencia de términos y la frecuencia inversa del documento.'''



model = TfidfVectorizer(stop_words='english')
womb = model.fit_transform(filter[groupData])

In [18]:
womb

<12x12 sparse matrix of type '<class 'numpy.float64'>'
	with 12 stored elements in Compressed Sparse Row format>

In [19]:
model

TfidfVectorizer(stop_words='english')

In [20]:
# Calculamos la similitud del coseno tomando todos las filas de la matriz
similarity = cosine_similarity(X = womb, Y = womb)
similarity.shape

(12, 12)

In [21]:
dfsimilarity = pd.DataFrame(similarity)
dfsimilarity

,0,1,2,3,4,5,6,7,8,9,10,11
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [22]:
dfsimilarity.to_parquet("Data/similarity")

In [23]:
# Reiniciamos el indice del dataframe y creamos una serie de índices
filter= filter.reset_index()
indice = pd.Series(filter.index, index=filter["item_id"])

In [24]:
filter.shape

(2699955, 14)

In [25]:
from tools import randomElim

In [26]:
filter = randomElim(filter, 2699000)

In [27]:
filter.shape

(955, 14)

In [28]:
filter.to_parquet("Data/modelFiltered")
indice.to_csv("Data/indexModel")

In [29]:
filter.head()

,index,item_id,item_name,playtime_forever,items_count,user_id,genres,price,developer,release_year,posted_year,recommend,sentiment_analysis,groupedata
3551,6233,91200,Anomaly Warzone Earth,16,524,thequeenpanda,['Action' 'Indie' 'Strategy'],9.99,11 bit studios,2011,2012,True,2,thequeenpanda Anomaly Warzone Earth 524 16 11 ...
12478,19800,407840,ORION,6,173,Xx-Woods,['Action' 'Indie' 'RPG'],0.99,"Trek Industries, Inc",2015,2015,True,2,"Xx-Woods ORION 173 6 Trek Industries, Inc ['Ac..."
12532,19910,220,Half-Life 2,1991,71,kevinator23,['Action'],9.99,Valve,2004,2012,True,2,kevinator23 Half-Life 2 71 1991 Valve ['Action...
13740,21999,331470,Everlasting Summer,47,45,A-Very-Sly-Person,['Adventure' 'Casual' 'Free to Play' 'Indie'],4.99,Soviet Games,2014,2014,True,2,A-Very-Sly-Person Everlasting Summer 45 47 Sov...
15828,26382,8870,BioShock Infinite,42,91,2768820078,['Action'],29.99,"Irrational Games,Aspyr (Mac),Virtual Programmi...",2013,2015,False,2,2768820078 BioShock Infinite 91 42 Irrational ...


In [30]:
indice.to_list()

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
